# Benchmark calculcations for Jan and June proposals

Assumptions:

- Buildings will produce the max amount of emissions, **unless** their predicted emissions is lower than the target GHGI, in which case they will produce that amount.
- Predicted emissions are calculated based off of the emissions predictions found in [the emissions data input file](../data/input_data/energy_emissions.csv). 



For each building, in each year (2027-2050), we need to calculate (column references given refer to the 2027 calculations in [the original RMI spreadsheet](https://docs.google.com/spreadsheets/d/175uipAHHQHGelq7i1n9sKWQXNQi-B1IJiF6-XQRVnE8/edit#gid=1811888818):

- `city_ghgi_target`: The GHGI target, as calculated by the sum of: use type * city's target GHGI for that use type * percent of GFA for that use type, for each of the three given use types (col A). If there is no target for any of the use types, this is NaN. If a building is multi-use, and some of the building's uses have a compliance threshold, but others don't, use the expected GHGI (greenhouse gas emissions intensity) if nothing is changed as the GHGI for the portion of the building that is not subject to BEPS yet.
    - Example: A building is 50% retail and 50% multifamily housing. The target for retail in 2033 is 1.03 and there is no target for multifamily housing. This building would have a GHGI of 4.0 in 2033 if no changes were made to it. We estimate the GHGHI target as `(0.5 * 1.03) + (0.5 * 4.0) = 2.515`. 
    - NB: These numbers will be different than the RMI calculations. The RMI model mistakenly used a GHGI of zero for parts of buildings that are not yet subject to BEPS. So in the example above, RMI's model would list the GHGI target as`0.5 * 1.03 + 0.5 * 0 = 0.515`, which leaves out half the building. Then the target GHGI would actually go **up** when the multifamily part of the building became subject to BEPS! This model corrects that error.
- `expected_baseline`: The expected emissions if nothing is changed about the building, as calculated by the sum of: `total use energy for type * energy emissions factor for energy type` for the three energy types (col C)
- `expected_baseline_ghgi`: The expected GHGI if nothing is changed about the building (col B), as calculated by the `expected emissions / total GFA`
- `compliant_ghgi`: The expected GHGI if the building is compliant with BEPS, as defined by (col H):
    - if the BEPS GHGI target is lower than the expected GHGI, use the BEPS GHGI target
    - if the expected GHGI is lower than the BEPS GHGI target, use the expected GHGI
- `compliant_emissions`: The expected emissions if the building is compliant with BEPS, as defined by the `compliant GHGI * total GFA` (col J)
- `compliance_status`: Whether or not the building is compliant (col K):
    - yes: the baseline GHGI is lower than the expected compliant GHGI for this year
    - no: the baseline GHGI is higher than the expected compliant GHGI for this year
    - no requirement yet: the building doesn't have a compliance requirement for this year
- `compliance_fees`: Noncompliance fees. For years where buildings will be taxed for being noncompliant, this is `$2.50 * total GFA`    

In [ ]:
# In which we discover an error in the RMI model
# The emissions for the multifamily housing part of this building get calculated as zero in the RMI model
# So when there is a GHGIT for multifamily housing, the building's GHGI will actually go *up*!
# No good
# Solution: this model will use the expected emissions if there are no changes to calculate the GHGI for the part
# of the building that is not yet subject to BEPS

# Yr: 2033
# Main Market, OSE ID 419
# sq ft: 77,640
# Retail Store	44,078 (57%)
# Restaurant	19,182 (25%)
# Multifamily Housing2	14,380 (19%)
# electric, steam, gas: 9,880,964	0	4,321,133

# electric, steam, gas emissions for 2033
# 2033	0.0026	0.0830	0.0530

# city GHGIs for 50-90k sq ft buildings in 2033
# retail: 1.03
# restaurant: 5.73
# multifamily: no target yet (RMI model gives 0)

# RMI model lists GHGIT as 2.0196
# show that uses 0 as the target for the multifamily part of the building
avg_ghgi = (.57 * 1.03) + (.25 * 5.73) + (.19 * 0)
avg_ghgi

In [1]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.mode.chained_assignment = None

## Load input data

Load input files:

- `energy_emissions`: the emissions factors for electric/steam/gas for each year, 2027-2050
- `scen_1_timeline`: the emissions benchmarks by building type and size in Scenario 1 (January proposal)
- `scen_2_timeline`: the emissions benchmarks by building type and size in Scenario 2 (June proposal)
- `building_data`: data for each >20k sq ft building in Seattle, including breakdown of building use percentage and baseline GHGI

In [2]:
def load_emissions(file_path):
    emissions = pd.read_csv(file_path)
    emissions.set_index('Year', inplace=True)
    return emissions

In [3]:
energy_emissions = load_emissions('../data/input_data/energy_emissions.csv')
scen_1_timeline = pd.read_csv('../data/input_data/scen_1_reformatted.csv')
scen_2_timeline = pd.read_csv('../data/input_data/scen_2_reformatted.csv')
building_data = pd.read_csv('../data/input_data/All-Benchmark-Data(2019)_reformatted.csv')

In [ ]:
energy_emissions.head()

In [6]:
scen_2_timeline.head()

,Unnamed: 0,year,building_type,sq_ft,sq_ft_classification,ghgi,max_size,min_size
0,0,2027,College/University,>220K Buildings,A,NaN,1000000,220000
1,1,2027,College/University,>90-220K Buildings,B,NaN,220000,90000
2,2,2027,College/University,>50-90K Buildings,C,NaN,90000,50000
3,3,2027,College/University,>30-50K Buildings,D,NaN,50000,30000
4,4,2027,College/University,>20-30K Buildings,E,NaN,50000,20000


In [ ]:
scen_2_timeline[(scen_2_timeline['year'] == 2027) & (scen_2_timeline['building_type'] == 'College/University') & (scen_2_timeline['sq_ft_classification'] == 'F')]

In [ ]:
building_data.head()

In [4]:
def classify_size(sq_ft):
    """
    Use letter classifications for building size instead of dealing with size ranges (>220k, 90-220k, etc.)
    """
    if sq_ft > 220000:
        return 'A'
    elif sq_ft > 90000:
        return 'B'
    elif sq_ft > 50000:
        return 'C'
    elif sq_ft > 30000:
        return 'D'
    elif sq_ft > 20000:
        return 'E'
    else:
        return 'F'

def clean_up_building_data(building_data, data_types):
    # Convert string numbers in "1,200" format to "1200". int("1,200") causes an error.
    for col in ['OSEBuildingID', 'PropertyGFATotal', 'PropertyGFABuilding(s)',
       'PropertyGFAParking', 'Total_sqft', 'LargestPropertyUseTypeGFA',
       'LargestPropertyUseTypeGFA Analysis', 'SecondLargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA Analysis', 'ThirdLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseTypeGFA Analysis', 'Electricity(kBtu)',
       'Steam(kBtu)', 'NaturalGas(kBtu)', 'TotalGHGEmissions', 'Total_GFA']:
        building_data[col] = building_data[col].apply(lambda x: x.replace(',', '')) 
    
    # Convert "95%" to 0.95
    for col in ['percent_sqft_1st', 'percent_sqft_2nd', 'percent_sqft_3rd']:
        building_data[col] = building_data[col].apply(lambda x: int(x.replace('%', '')) / 100.0)
    
    # Cast columns in correct data types
    building_data = building_data.astype(data_types)
    
    # Classify building size
    building_data['sq_ft_classification'] = building_data['Total_sqft'].apply(lambda x: classify_size(x))
    
    # Fix weird typo where Multifamily housing has been replaced with Multifamily Housing2, 3, etc.
    for col in ['LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE']:
        building_data[col] = building_data[col].replace(regex=r'Multifamily Housing[0-9]', value='Multifamily Housing')
    
    return building_data

In [5]:
building_data_types = {
    'Unnamed: 0': 'str', 
    'OSEBuildingID': 'int', 
    'BuildingName': 'str', 
    'BuildingType': 'str',
    'Type_of_Bulding': 'str', 
    'PropertyGFATotal': 'int', 
    'PropertyGFABuilding(s)': 'int',
    'PropertyGFAParking': 'int', 
    'Total_sqft': 'int', 
    'percent_sqft_1st': 'float64',
    'percent_sqft_2nd': 'float64', 
    'percent_sqft_3rd': 'float64', 
    'LargestPropertyUseType': 'str',
    'LargestPropertyUseType OSE': 'str', 
    'LargestPropertyUseTypeGFA': 'int',
    'LargestPropertyUseTypeGFA Analysis': 'int', 
    'SecondLargestPropertyUseType': 'str',
    'SecondLargestPropertyUseType OSE': 'str', 
    'SecondLargestPropertyUseTypeGFA': 'int',
    'SecondLargestPropertyUseTypeGFA Analysis': 'int',
    'ThirdLargestPropertyUseType': 'str', 
    'ThirdLargestPropertyUseType OSE': 'str',
    'ThirdLargestPropertyUseTypeGFA': 'int',
    'ThirdLargestPropertyUseTypeGFA Analysis': 'int', 
    'Electricity(kBtu)': 'int',
    'Steam(kBtu)': 'int', 
    'NaturalGas(kBtu)': 'int', 
    'TotalGHGEmissions': 'float64',
    'GHGEmissionsIntensity': 'float64', 
    'Total_GFA': 'int'
}

In [ ]:
building_data = clean_up_building_data(building_data, building_data_types)

In [ ]:
building_data.head()

In [ ]:
class BenchmarkModel:
    def self.__init__(timeline, building_data, fine_years, energy_emissions):
        self.timeline = timeline
        self.building_data = building_data
        self.energy_emissions = energy_emissions 
        self.fine_years = fine_years # array of years when building owners will be fined
    
    def find_ghgi_standard(year, building_type, sq_ft_class):
        '''
        Find the GHGI standard in the model's timeline for a given year, building type, and building size.
        '''
        # building types listed as NAN don't count towards the policy, so their emissions and sq ft are calculated as zero
        if pd.isna(building_type) or building_type == 'nan':
            return 0
        row = self.timeline[(self.timeline['year'] == year) & (self.timeline['sq_ft_classification'] == sq_ft_class) & (self.timeline['building_type'] == building_type)]
        return row.iloc[0]['ghgi'] 
    
    def get_expected_baseline(building, year):
        '''
        Find the expected baseline GHGE for a given year if the building makes no changes
        '''
        electric_emissions = building['Electricity(kBtu)'] * self.energy_emissions.loc[year]['Electricity emission factor (kgCO2e/kBtu)']
        steam_emissions = building['Steam(kBtu)'] * self.energy_emissions.loc[year]['Steam emission factor (kgCO2e/kBtu)']
        gas_emissions = building['NaturalGas(kBtu)'] * self.energy_emissions.loc[year]['Gas emission factor (kgCO2e/kBtu)']
        return electric_emissions + steam_emissions + gas_emissions
    
    def get_expected_baseline_ghgi(building):
        '''
        Return the expected GHGI for a bulding if the building makes no changes
        '''
        if building['Total_sqft'] == 0:
            return 0

        return building['expected_baseline'] / building['Total_sqft']
    
    def fill_in_expected_baselines(year_low, year_high, input_df, output_df):
        '''
        input_df: dataframe without any calculations, only building data
        output_df: df with OSE ID, building name, total sq ft, sq ft classification, year, and expected baseline for each year in between year_low and year_high
        '''
        for year in range(year_low, year_high + 1):
            temp_df = input_df[['OSEBuildingID', 'BuildingName', 'Total_sqft', 'sq_ft_classification', 'LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE']]
            temp_df['year'] = pd.Series([year]*len(temp_df))
            temp_df['expected_baseline'] = input_df.apply(lambda building: self.get_expected_baseline(building, year), axis=1)

            output_df = pd.concat([output_df, temp_df])


        return output_df
    
    def get_city_ghgi(building, orig_building_data):
        year = building['year']
        baseline_ghgi = building['expected_baseline_ghgi']
        orig_building_row = self.building_data[self.building_data['OSEBuildingID'] == building['OSEBuildingID']].iloc[0]

        first_use_ghgi = self.find_ghgi_standard(timeline, year, orig_building_row['LargestPropertyUseType OSE'], building['sq_ft_classification'])
        second_use_ghgi = self.find_ghgi_standard(timeline, year, orig_building_row['SecondLargestPropertyUseType OSE'], building['sq_ft_classification'])
        third_use_ghgi = self.find_ghgi_standard(timeline, year, orig_building_row['ThirdLargestPropertyUseType OSE'], building['sq_ft_classification'])

        return (orig_building_row['percent_sqft_1st'] * (baseline_ghgi if pd.isna(first_use_ghgi) else first_use_ghgi)) + (orig_building_row['percent_sqft_2nd'] * (baseline_ghgi if pd.isna(second_use_ghgi) else second_use_ghgi)) + (orig_building_row['percent_sqft_3rd'] * (baseline_ghgi if pd.isna(third_use_ghgi) else third_use_ghgi))

    def get_compliant_ghgi(building):
        '''
        Return the lower of: the city's benchmark and the expected emissions for a building
        '''
        return min(building['expected_baseline_ghgi'], building['city_ghgi_target'])

    def get_compliant_emissions(building):
        return building['compliant_ghgi'] * building['Total_sqft']

    def get_compliance_status(building):
        year = building['year']
        sqft_class = building['sq_ft_classification']
        if pd.isna(self.find_ghgi_standard(year, building['LargestPropertyUseType OSE'], sqft_class)) and pd.isna(find_ghgi_standard(timeline, year, building['SecondLargestPropertyUseType OSE'], sqft_class)) and pd.isna(find_ghgi_standard(timeline, year, building['ThirdLargestPropertyUseType OSE'], sqft_class)): 
            return 'Not due yet'
        elif building['expected_baseline_ghgi'] < building['city_ghgi_target']:
            return 'Yes'
        else:
            return 'No'
        
    def get_noncompliance_fines(building):
        if building['year'] in self.fines_years:
            return building['Total_sqft'] * 2.5
        else:
            return 0
    
    # bring everything together
    def calculate_baseline_model():
        

In [ ]:
# helpers to find data

def find_ghgi_standard(timeline, year, building_type, sq_ft_class):
    if pd.isna(building_type) or building_type == 'nan':
        return 0
    row = timeline[(timeline['year'] == year) & (timeline['sq_ft_classification'] == sq_ft_class) & (timeline['building_type'] == building_type)] # & (timeline['building_type'] == building_type)
    return row.iloc[0]['ghgi']

# helpers to calculate columns

def get_expected_baseline(building, year):
    electric_emissions = building['Electricity(kBtu)'] * energy_emissions.loc[year]['Electricity emission factor (kgCO2e/kBtu)']
    steam_emissions = building['Steam(kBtu)'] * energy_emissions.loc[year]['Steam emission factor (kgCO2e/kBtu)']
    gas_emissions = building['NaturalGas(kBtu)'] * energy_emissions.loc[year]['Gas emission factor (kgCO2e/kBtu)']
    return electric_emissions + steam_emissions + gas_emissions

def fill_in_expected_baselines(year_low, year_high, input_df, output_df):
    '''
    input_df: dataframe without any calculations, only building data
    output_df: df with OSE ID, building name, total sq ft, sq ft classification, year, and expected baseline
    '''
    for year in range(year_low, year_high + 1):
        temp_df = input_df[['OSEBuildingID', 'BuildingName', 'Total_sqft', 'sq_ft_classification', 'LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE']]
        temp_df['year'] = pd.Series([year]*len(temp_df))
        temp_df['expected_baseline'] = input_df.apply(lambda building: get_expected_baseline(building, year), axis=1)

        output_df = pd.concat([output_df, temp_df])

    
    return output_df

def get_expected_baseline_ghgi(building):
    if building['Total_sqft'] == 0:
        return 0
    
    return building['expected_baseline'] / building['Total_sqft']

def get_city_ghgi(building, timeline, orig_building_data):
    year = building['year']
    baseline_ghgi = building['expected_baseline_ghgi']
    orig_building_row = orig_building_data[orig_building_data['OSEBuildingID'] == building['OSEBuildingID']].iloc[0]
    
    first_use_ghgi = find_ghgi_standard(timeline, year, orig_building_row['LargestPropertyUseType OSE'], building['sq_ft_classification'])
    second_use_ghgi = find_ghgi_standard(timeline, year, orig_building_row['SecondLargestPropertyUseType OSE'], building['sq_ft_classification'])
    third_use_ghgi = find_ghgi_standard(timeline, year, orig_building_row['ThirdLargestPropertyUseType OSE'], building['sq_ft_classification'])
    
    return (orig_building_row['percent_sqft_1st'] * (baseline_ghgi if pd.isna(first_use_ghgi) else first_use_ghgi)) + (orig_building_row['percent_sqft_2nd'] * (baseline_ghgi if pd.isna(second_use_ghgi) else second_use_ghgi)) + (orig_building_row['percent_sqft_3rd'] * (baseline_ghgi if pd.isna(third_use_ghgi) else third_use_ghgi))

def get_compliant_ghgi(building):
    return min(building['expected_baseline_ghgi'], building['city_ghgi_target'])

def get_compliant_emissions(building):
    return building['compliant_ghgi'] * building['Total_sqft']

def get_compliance_status(building, timeline):
    year = building['year']
    sqft_class = building['sq_ft_classification']
    if pd.isna(find_ghgi_standard(timeline, year, building['LargestPropertyUseType OSE'], sqft_class)) and pd.isna(find_ghgi_standard(timeline, year, building['SecondLargestPropertyUseType OSE'], sqft_class)) and pd.isna(find_ghgi_standard(timeline, year, building['ThirdLargestPropertyUseType OSE'], sqft_class)): 
        return 'Not due yet'
    elif building['expected_baseline_ghgi'] < building['city_ghgi_target']:
        return 'Yes'
    else:
        return 'No'

scen_2_fines_years = [2035, 2040, 2045, 2050]
def get_noncompliance_fines(building, fines_years):
    if building['year'] in fines_years:
        return building['Total_sqft'] * 2.5
    else:
        return 0

## Calculate GHGIs and compliance status

In [ ]:
base_building_info = pd.DataFrame(columns=['OSEBuildingID', 'BuildingName', 'Total_sqft', 'sq_ft_classification', 'LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE'])

In [ ]:
# scen_2_calcs.to_csv('scen_2_calcs.csv')

In [ ]:
# scen_2_calcs = pd.read_csv('scen_2_calcs.csv')

In [ ]:
scen_2_timeline['building_type']= scen_2_timeline['building_type'].replace(regex=r'Multifamily Housing[0-9]', value='Multifamily Housing')

In [ ]:
for col in ['LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE']:
    scen_2_calcs[col] = building_data[col].replace(regex=r'Multifamily Housing[0-9]', value='Multifamily Housing')

In [ ]:
scen_2_calcs['SecondLargestPropertyUseType OSE'].unique()

In [ ]:
def get_scen_calcs(orig_building_data, timeline, start_year, end_year):
    base_building_info = pd.DataFrame(columns=['OSEBuildingID', 'BuildingName', 'Total_sqft', 'sq_ft_classification'])
    scen_calcs = fill_in_expected_baselines(start_year, end_year, orig_building_data, base_building_info)
    
    scen_calcs['expected_baseline_ghgi'] = scen_calcs.apply(lambda building: get_expected_baseline_ghgi(building), axis=1)
    
    scen_calcs['city_ghgi_target'] = scen_calcs.apply(lambda building: get_city_ghgi(building, timeline, orig_building_data), axis=1)
    
    return scen_calcs

In [ ]:
energy_emissions.set_index('Year', inplace=True)
scen_2_calcs = fill_in_expected_baselines(2027, 2050, building_data, base_building_info)

In [ ]:
scen_2_calcs['expected_baseline_ghgi'] = scen_2_calcs.apply(lambda building: get_expected_baseline_ghgi(building), axis=1)

In [ ]:
scen_2_calcs['city_ghgi_target'] = scen_2_calcs.apply(lambda building: get_city_ghgi(building, scen_2_timeline, building_data), axis=1)

In [ ]:
scen_2_calcs.head()

In [ ]:
scen_2_timeline['building_type']= scen_2_timeline['building_type'].replace(regex=r'Multifamily Housing[0-9]', value='Multifamily Housing')

In [ ]:
scen_2_calcs['LargestPropertyUseType OSE'].unique()

In [ ]:
scen_2_calcs.head()

In [ ]:
scen_2_calcs['city_ghgi_target'] = scen_2_calcs.apply(lambda building: get_city_ghgi(building, scen_2_timeline, building_data), axis=1)

In [ ]:
scen_2_calcs.head()

In [ ]:
scen_2_calcs.columns

In [ ]:
scen_2_calcs['compliant_ghgi'] = scen_2_calcs.apply(lambda building: get_compliant_ghgi(building), axis=1)

In [ ]:
scen_2_calcs[100:120]

In [ ]:
scen_2_calcs['compliant_emissions'] = scen_2_calcs.apply(lambda building: get_compliant_emissions(building), axis=1)

In [ ]:
scen_2_calcs['compliance_status'] = scen_2_calcs.apply(lambda building: get_compliance_status(building, scen_2_timeline), axis=1)

In [ ]:
scen_2_fines_years = [2035, 2040, 2045, 2050]

scen_2_calcs['compliance_fees'] = scen_2_calcs.apply(lambda building: get_noncompliance_fines(building, scen_2_fines_years), axis=1)

In [ ]:
scen_2_calcs.to_csv('scen_2_calcs.csv')

In [ ]:
next:
    - run model for scenario 2, then scenario 1
    - alternate compliance options

## Scenario 1 (Jan.)

In [ ]:
scen_1_timeline = pd.read_csv('../data/input_data/scen_1_reformatted.csv')

In [ ]:
scen_1_timeline['building_type'] = scen_1_timeline['building_type'].replace(r'Multifamily Housing[0-9]')

In [ ]:
scen_1_calcs = get_scen_calcs(building_data, scen_1_timeline, 2027, 2050)

In [ ]:
scen_1_calcs.head()

In [ ]:
scen_1_calcs['compliant_ghgi'] = scen_1_calcs.apply(lambda building: get_compliant_ghgi(building), axis=1)

In [ ]:
scen_1_calcs['compliant_emissions'] = scen_1_calcs.apply(lambda building: get_compliant_emissions(building), axis=1)

In [ ]:
scen_1_calcs['compliance_status'] = scen_1_calcs.apply(lambda building: get_compliance_status(building, scen_1_timeline), axis=1)

In [ ]:
scen_1_fines_years = [2030, 2035, 2040, 2045, 2050]
scen_1_calcs['compliance_fees'] = scen_1_calcs.apply(lambda building: get_noncompliance_fines(building, scen_1_fines_years), axis=1)

In [ ]:
scen_1_calcs.to_csv('scen_1_calcs.csv')

In [ ]:
## Summary stats

In [ ]:
def get_total_emissions(df):
    
    # 2026 is just predicted baseline for 2027
    #emissions_2026 = df[df['year'] == 2027]['expected_baseline_ghgi'].sum()
    
    grouped = df[df['sq_ft_classification'] != 'F'].groupby('year')['compliant_emissions'].sum()
    grouped = grouped.to_frame()
    
    #grouped.loc[2026] = df[df['year'] == 2027]['expected_baseline_ghgi'].sum()
    
    return grouped

In [ ]:
scen_1_emissions = get_total_emissions(scen_1_calcs)

scen_1_emissions.to_csv('scen_1_emissions.csv')

In [ ]:
scen_1_emissions

In [ ]:
scen_2_emissions = get_total_emissions(scen_2_calcs)

scen_2_emissions.to_csv('scen_2_emissions.csv')

In [ ]:
building_data.to_csv('building_data_clean.csv')